In [1]:
!pip install -q transformers torchaudio librosa pyannote.audio gradio

In [2]:
# Login to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# 📁 Step 2: Upload Audio File
from google.colab import files
uploaded = files.upload()


Saving test_voice.mp3 to test_voice (1).mp3


In [4]:
# 🧠 Step 3: Load Whisper ASR Model
from transformers import pipeline
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [5]:
# 📥 Step 4: Transcribe Audio
import torchaudio

file_path = list(uploaded.keys())[0]
waveform, sample_rate = torchaudio.load(file_path)
transcription = asr(file_path, chunk_length_s=30, stride_length_s=(5,2))
print("✅ Transcription Done!\n")
print(transcription["text"])

Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


✅ Transcription Done!

 Sarah, project lead, good morning team. Our customer support volume is soaring. I'm proposing Project Apollo and AI chatbot to handle routine inquiries, reducing human agent workload by 25% and improving customer satisfaction. David, technical architect, will use NLP and NGPT4 architecture. Find tuning it with our data. Phase one is 12 weeks, requiring dedicated AIML resources and close support team collaboration. is accurate, understands user intent, and has seamless human handoffs. Sarah, share the detailed plan and budget. Next tech in in two weeks.


In [6]:
# 🔍 Step 5: Keyword Search
def search_keywords(transcript, keywords):
    results = []
    for keyword in keywords:
        if keyword.lower() in transcript.lower():
            results.append(f"✅ Found '{keyword}'")
        else:
            results.append(f"❌ Not Found '{keyword}'")
    return results

In [7]:
# Example
keywords = ["duration", "project", "budget"]
print(search_keywords(transcription["text"], keywords))


["❌ Not Found 'duration'", "✅ Found 'project'", "✅ Found 'budget'"]


In [8]:
# 👤 Step 6: Speaker Diarization (Optional but Powerful)
from pyannote.audio import Pipeline


In [9]:
# This model requires you to accept usage terms on HF
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/hyperparams.yaml'


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load_if_possible
DEBUG:speechbrain.utils.parameter_transfer:Collecting files (or symlinks) for pretraining in /root/.cache/torch/pyannote/speechbrain.
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["embedding_model"] = /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Using symlink found at '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Using symlink found at '/root/.cache/

In [ ]:
# Diarization
diarization = pipeline(file_path)
print("\n🗣️ Speaker Segments:\n")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{speaker} speaks from {turn.start:.1f}s to {turn.end:.1f}s")

In [11]:
# 📌 (Optional) Visualize with Gradio
import gradio as gr

def keyword_search_ui(keywords):
    result = search_keywords(transcription["text"], keywords.split(","))
    return "\n".join(result)


In [15]:
gr.Interface(fn=keyword_search_ui,
             inputs="text",
             outputs="text",
             title="Keyword Search in Audio Transcript").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://26a3f2710ac3eacb24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
